In [1]:
import os

In [2]:
%pwd

'c:\\Users\\RadhikaMaheshwari\\Desktop\\Test\\DeepLearning\\ETE\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\RadhikaMaheshwari\\Desktop\\Test\\DeepLearning\\ETE'

In [5]:
# Step 30: We first need to update config.yaml file
# We need to add model training related configuration. 
# Reason why we have config.yaml file is that just in case we need to update any directory name or file path or 
# any other config in future then we will not need to go to component, pipeline or any other place to update them. 
# We will only need to update the info here and it will be reflected in all the places. 

In [6]:
# For model trainer config: We are defining the root directory where all the data and steps for model trainer will be
# stored. Then we are defining the train and test data paths that will be required to train and test model.
# Then we are defining model name using joblib
# Step 30 complted

In [7]:
# Step 31: Now we need to define params as we have already chosen the model so we can define params for that. 
# This will be done in Params.yaml file. Now we are using elastic net so we will be defining params related to that. 
# Step 31 completed

In [8]:
# Step 32: Now we need to update the entity. 
# In this we are not only returning the variables from the config.yaml file instead we are also returning our parameters from 
# the params.yaml file. Also, we are storing target column from the schema.yaml file. 
from dataclasses import dataclass 
from pathlib import Path 

@dataclass(frozen= True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path 
    test_data_path: Path 
    model_name: str 
    alpha: float 
    l1_ratio: float 
    target_column: str 

In [9]:
# Step 32 completed. Now we need to update our configuration manager

In [10]:
# Step 33: Updating configuration manager

In [11]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [12]:
class ConfigurationManager:
    def __init__(self,
                config_filepath = CONFIG_FILE_PATH,
                params_filepath = PARAMS_FILE_PATH,
                schema_filepath = SCHEMA_FILE_PATH
                 ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path = config.test_data_path,
            model_name=config.model_name,
            alpha = params.alpha,
            l1_ratio = params.l1_ratio,
            target_column=schema.name
        )
        return model_trainer_config


In [13]:
# Step 33 completed

In [14]:
# Step 34: Update components


In [15]:
import pandas as pd
import os 
from mlProject import logger
from sklearn.linear_model import ElasticNet
import joblib

In [16]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config 
    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        train_x = train_data.drop([self.config.target_column], axis = 1)
        train_y = train_data[[self.config.target_column]]
        test_x = test_data.drop([self.config.target_column], axis = 1)
        test_y = test_data[[self.config.target_column]]

        lr = ElasticNet(alpha = self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))
    

In [17]:
# Step 34 Completed

In [18]:
# Step 35: Now we need to update our pipeline

In [19]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config= model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-05-28 15:17:18,635: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-28 15:17:18,646: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-28 15:17:18,649: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-05-28 15:17:18,651: INFO: common: Created Directory at: artifacts]
[2024-05-28 15:17:18,652: INFO: common: Created Directory at: artifacts/model_trainer]


In [20]:
# Step 35 completed. We will now convert this notebook to modular coding